In [7]:
import pandas as pd
import numpy as np 
import configparser 
import pymysql
from tqdm import tqdm
import requests
import re
import random
from resume_text import *

In [8]:
get_user_resume_text(1809049)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5090.17it/s]


'CA KRISHAN GAUTAM kkgautam91@gmail.com +91-9811256768 SKILL SET Financial Planning and Analysis Accounts Management Audits, Due Diligence Finance, Credit control Tax compliance &amp; Planning Payroll accounting Ind-AS SAP (FICO Module) PROFILE SNAPSHOT A finance and accounts professional (CA) with 7+ year of experience Rich experience of accounting &amp; finance operations i.e. preparation and maintenance of statutory books of accounts and finalization of them at the year end. Track record of overseeing Financial Statements including Trial Balance, P&amp;L, Balance Sheet, Age wise account receivable and payables. Gained exposure of Audits comprising Tax Audit, Statutory audit, internal audit and due diligence. Relevant experience of funding through bank in terms of cash credit, term loans, letter of credit management. Projects, foreign remittance, liaising with stakeholders. EDUCATIONAL DETAILS Qualification YEAR Chartered Accountant May, 2015 B. Com. April, 2010 Higher Secondary Scho

In [2]:
pwd

'/home/ubuntu/infoedge/llama-recipes/examples'

In [3]:
config = configparser.ConfigParser()

config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [4]:
 

mydb= pymysql.connect(

host=config['iimjobs_db']['hostname'],

user=config['iimjobs_db']['username'],

password=config['iimjobs_db']['password'],

db=config['iimjobs_db']['db_name']

)

 

mycursor = mydb.cursor()

 

In [5]:
query = "SELECT id FROM profile_info WHERE last_active_date > '2023-12-03' ORDER BY last_active_date DESC limit 200"

 

mycursor.execute(query)

result = mycursor.fetchall()

In [6]:
uid_df=pd.DataFrame(result,columns=['userid'])

In [7]:
# user_df[['userid']]

In [8]:
# user_df['userid']

In [9]:
uid_list =list(uid_df['userid'])


In [10]:
random.choice(uid_list)

1809049

In [11]:
random.sample(uid_list,5)

[1250581, 2779605, 2550205, 2764347, 493341]

In [29]:

 
 
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)
    
    return raw_data

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'resume' : [],
        'current_designation' : [],
        'user_experience':[],
        'professional_info':[],
        'education_info':[]
        # 'company_designation':[],
        # 'company_name':[]
        
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(source['resumeText'])
            user_dict['current_designation'].append(source['currentDesignation'])
            user_dict['user_experience'].append(source['expYear'])
            user_dict['professional_info'].append(source['professionalInfo'])
            user_dict['education_info'].append(source['educationInfo'])            

        except Exception as e:
            continue

    return user_dict

In [10]:
# get_user_data_search_embed([2550205])

In [13]:
user_df = get_user_data_search_embed(random.sample(uid_list,5))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 94466.31it/s]


In [14]:
user_df

,id,resume,current_designation,user_experience,professional_info,company_designation,company_name
0,698334,RAMA SAI SANDILYA KURAPATI\nMobile: +91-735851...,Sr. Product Manager,7,"[{'id': 795940, 'userId': None, 'designation':...",Sr. Product Manager,Koo India
1,1859213,"Yogendra Ramanuj Pandey\nC-501, Ashok Smruti, ...",Chief Manager-Taxation,18,"[{'id': 4175062, 'userId': None, 'designation'...",Chief Manager-Taxation,Koo India
2,190807,"Anurag Khan\nDate of Birth: 22nd\nAugust, 1986...",Cluster Head - Business Lending (SME Coverage),11,"[{'id': 269769, 'userId': None, 'designation':...",Cluster Head - Business Lending (SME Coverage),Times Group
3,344369,KAUSTUBH KOLHAR\nAge: 28 years | Email: kaustu...,Relationship Manager,5,"[{'id': 205427, 'userId': None, 'designation':...",Relationship Manager,Times Group
4,,,,,,,IDFC
5,,,,,,,IDFC
6,,,,,,,ICICI Bank
7,,,,,,,ICICI Bank


In [5]:
# clean_space(user_df['resume'].values[0])

In [6]:
# t = get_user_resume_text([907200])


In [7]:
# t

<!-- premium user iimjobs 2550205 -->

In [13]:
df = pd.read_csv('custom_data/latest_work_exp_28dec.csv')

In [29]:
df[['work_experience_new']].sample().values[0][0]

"[{'company': 'HCL Technology', 'role': 'Senior Software Engineer', 'start_date': 'Dec 2021', 'end_date': 'Present', 'description': None}, {'company': 'Tata Consultancy Services (TCS)', 'role': 'Software Engineer (Frontend UI Developer)', 'start_date': 'February 2020', 'end_date': 'Dec 2021', 'description': None}, {'company': 'SRM Techsol Pvt Ltd', 'role': 'Software Engineer (Full Stack Developer)', 'start_date': 'April 2017', 'end_date': 'January 2020', 'description': None}, {'company': 'Cedcoss Technology Pvt Ltd', 'role': 'Software Trainee (Magento Developer)', 'start_date': 'May 2016', 'end_date': 'Nov 2016', 'description': None}]"

In [1]:
from user_info import *

In [9]:
get_user_info(2550205)['education_info']

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16008.79it/s]


[[{'id': 97442960,
   'userId': 2550205,
   'institute': {'id': 1,
    'name': 'Indian Institute of Management, Ahmedabad (IIMA)',
    'nameNotAnalyzed': 'Indian Institute of Management, Ahmedabad (IIMA)'},
   'batchFrom': 2020,
   'batchTo': 2022,
   'isPrimary': 1,
   'batch': '',
   'courseType': 1,
   'degree': {'id': 1, 'name': 'MBA / PGDM', 'nameNotAnalyzed': 'MBA / PGDM'},
   'rank': 0,
   'attempt': 0,
   'dateCreated': 1682965800000,
   'dateUpdated': 1685385000000,
   'status': 1,
   'isTopInstitute': 1,
   'isSuperTopInstitute': 1,
   'shouldTopInstituteBoost': 1,
   'shouldSuperTopInstituteBoost': 0},
  {'id': 97442959,
   'userId': 2550205,
   'institute': {'id': 164,
    'name': 'National Institute of Technology, Kurukshetra (NIT KKR)',
    'nameNotAnalyzed': 'National Institute of Technology, Kurukshetra (NIT KKR)'},
   'batchFrom': 2011,
   'batchTo': 2015,
   'isPrimary': 0,
   'batch': '',
   'courseType': 1,
   'degree': {'id': 6,
    'name': 'B Tech / BE',
    'name